In [ ]:
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
# from pykrige.rk import RegressionKriging
import geopandas as gpd
import rasterio
import rioxarray
import pdal

In [ ]:
# load the bathy points

pts_gdf_all = gpd.read_file('../data/test_sites/florida_keys/all_bathy_pts.gpkg')


In [ ]:
pts_gdf_all

In [ ]:
# # create a numpy array that PDAL can read by subsetting the columns and renaming the northing, easting, etc columns to the LAS defaults (Z,Y,Z)
pdal_array = pts_gdf_all.loc[:,['northing','easting','z_kde']].rename(columns={'northing':'Y','easting':'X','z_kde':'Z'}).to_records(index=False)

# pipeline = pdal.Filter.elm(cell=5).pipeline(pdal_array)
# pipeline |= pdal.Writer.las(filename='../data/resample_test/all_pts.laz')

# print(pipeline.execute())

In [ ]:

# create a PDAL pipeline
pipeline = pdal.Filter.relaxationdartthrowing(count=8000).pipeline(pdal_array)

print(pipeline.execute())

thinned_array = pipeline.arrays[0]

pipeline = pdal.Writer.las(filename='../data/resample_test/filtered.laz').pipeline(thinned_array)
print(pipeline.execute())

In [ ]:
pts_gdf = gpd.GeoDataFrame(thinned_array,geometry=gpd.points_from_xy(thinned_array['X'],thinned_array['Y'],crs='EPSG:32617'))

In [ ]:
# keep points within a half meter of the estimated seafloor
# pts_gdf = pts_gdf[abs(pts_gdf.Z_g - pts_gdf.z_kde) < 0.005]

In [ ]:
def krige_bathy(pts_gdf,krmodel):
    # pts_gdf = pts_gdf.sort_values('kde_val',ascending=False).iloc[:3000]
    ras = rasterio.open('../data/resample_test/bilinear.tif')
    ar = rioxarray.open_rasterio(ras)
    gridx = ar.x.data
    gridy = ar.y.data
    # going to randomly downsample the lidar point dataframe to make the UK model faster

    # pts_gdf = pts_gdf.sample(1000)
    pts_gdf.to_file('../data/resample_test/keys_testpts.gpkg')

    x_loc = pts_gdf.geometry.x.to_numpy()
    y_loc = pts_gdf.geometry.y.to_numpy()
    z_elev = pts_gdf.Z.to_numpy()
    krigemodel = krmodel(
        x=x_loc,
        y=y_loc,
        z=z_elev,
        variogram_model='linear',
        verbose=True,
        coordinates_type = "euclidean",
    )

    # krigemodel = UniversalKriging(
    #     x=x_loc,
    #     y=y_loc,
    #     z=z_elev,
    #     variogram_model='linear',
    #     verbose=True,
    #     # coordinates_type = "euclidean",
    # )
        
    z,ss = krigemodel.execute('grid',gridx,gridy)
    # xarray/rioxarray are too complicated, going to just use rasterio

    with rasterio.open('../data/resample_test/interp_OK.tif',mode='w+',crs=ras.crs,width=ras.width,height=ras.height,count=2,dtype=ras.dtypes[0],transform=ras.transform,) as rasout:
        rasout.write(z,1)
        rasout.write(ss,2)


krige_bathy(pts_gdf=pts_gdf,krmodel=OrdinaryKriging)